In [60]:
import os
from PIL import Image, ImageFont, ImageDraw

if not os.path.exists('C:/Users/MANUEL/Desktop/handwritten/habbakuk/'):
        os.makedirs('C:/Users/MANUEL/Desktop/handwritten/habbakuk/')


path_pretrain_habbakuk="C:/Users/MANUEL/Desktop/handwritten/habbakuk"
#Load the font and set the font size to 42
font = ImageFont.truetype('C:/Users/MANUEL/Desktop/handwritten/Habbakuk.ttf', 42)

#Character mapping for each of the 27 tokens
char_map = {'Alef' : ')', 
            'Ayin' : '(', 
            'Bet' : 'b', 
            'Dalet' : 'd', 
            'Gimel' : 'g', 
            'He' : 'x', 
            'Het' : 'h', 
            'Kaf' : 'k', 
            'Kaf-final' : '\\', 
            'Lamed' : 'l', 
            'Mem' : '{', 
            'Mem-medial' : 'm', 
            'Nun-final' : '}', 
            'Nun-medial' : 'n', 
            'Pe' : 'p', 
            'Pe-final' : 'v', 
            'Qof' : 'q', 
            'Resh' : 'r', 
            'Samekh' : 's', 
            'Shin' : '$', 
            'Taw' : 't', 
            'Tet' : '+', 
            'Tsadi-final' : 'j', 
            'Tsadi-medial' : 'c', 
            'Waw' : 'w', 
            'Yod' : 'y', 
            'Zayin' : 'z'}

#Returns a grayscale image based on specified label of img_size
def create_image(label, img_size):
    if (label not in char_map):
        raise KeyError('Unknown label!')

    #Create blank image and create a draw interface
    img = Image.new('L', img_size, 255)    
    draw = ImageDraw.Draw(img)

    #Get size of the font and draw the token in the center of the blank image
    w,h = font.getsize(char_map[label])
    draw.text(((img_size[0]-w)/2, (img_size[1]-h)/2), char_map[label], 0, font)

    return img

#Creates a 50x50 images of all the hebrew alphabet and save it to disk
#To get the raw data cast it to a numpy array

for character in list(char_map.keys()):
    img = create_image(character, (50, 50))
    os.mkdir(os.path.join(path, character))
    img.save(f'C:/Users/MANUEL/Desktop/handwritten/habbakuk/{character}/example_{character}.png')


In [163]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [164]:
batch_size=12
input_size = (64, 64)
optimizer = 'adam'

In [165]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [166]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [167]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
image_shape = (64,64,3)
np.random.seed(1000)
model = Sequential()
#First Convolutional layer
model.add(Conv2D(filters=6, input_shape=image_shape, kernel_size=(11,11), strides=(1,1),padding='valid'))
model.add(Activation('relu'))

#Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

#Second Convolutional layer
model.add(Conv2D(filters=6, kernel_size=(5,5), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

#Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

#Third Convolutional layer
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

#Fourth Convolutional layer
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

#Fifth Convolutional layer
model.add(Conv2D(filters=22, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

#Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

#Passing it to a fully connected layer, here we do the flatten!
model.add(Flatten())

#First Fully Connected layer has 4096 neurons
model.add(Dense(4096, input_shape=(227*227*3,)))
model.add(Activation('relu'))

#Add dropout to prevent overfitting
model.add(Dropout(0.4))

#Second Fully Connected layer
model.add(Dense(4096))
model.add(Activation('relu'))

#Add Dropout
model.add(Dropout(0.4))

#Output layer
model.add(Dense(27))
model.add(Activation('softmax'))

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])



Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 54, 54, 6)         2184      
_________________________________________________________________
activation_24 (Activation)   (None, 54, 54, 6)         0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 26, 26, 6)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 22, 22, 6)         906       
_________________________________________________________________
activation_25 (Activation)   (None, 22, 22, 6)         0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 10, 10, 6)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 8, 8, 16)         

In [168]:
gen_train = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1,
        shear_range=0.15, zoom_range=0.1,
        channel_shift_range=10., horizontal_flip=False)

In [169]:
gen_test = ImageDataGenerator(rescale=1./255, rotation_range=10)

In [171]:
train_generator = gen_train.flow_from_directory(
                path_pretrain_habbakuk, 
                target_size=input_size, 
                batch_size=batch_size,
                class_mode='categorical',subset='training')

Found 27 images belonging to 27 classes.


In [170]:
validation_generator = gen_test.flow_from_directory(
                path_pretrain_habbakuk,  
                target_size=input_size,
                batch_size=batch_size,
                class_mode='categorical', subset='validation')

Found 0 images belonging to 27 classes.


In [172]:
weights_pretrain = model.fit_generator(
        train_generator,
        steps_per_epoch= 27 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=27 // batch_size)
model.save_weights(f'{character}+.h5')

Epoch 1/50
2/2 [==============================] - 3s 16ms/step - loss: 3.3139 - accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 0s 18ms/step - loss: 3.3379 - accuracy: 0.0444
Epoch 3/50
2/2 [==============================] - 0s 16ms/step - loss: 3.3004 - accuracy: 0.0722
Epoch 4/50
2/2 [==============================] - 0s 28ms/step - loss: 3.2932 - accuracy: 0.0556
Epoch 5/50
2/2 [==============================] - 0s 26ms/step - loss: 3.3055 - accuracy: 0.0444
Epoch 6/50
2/2 [==============================] - 0s 17ms/step - loss: 3.2918 - accuracy: 0.0000e+00
Epoch 7/50
2/2 [==============================] - 0s 25ms/step - loss: 3.2975 - accuracy: 0.0278
Epoch 8/50
2/2 [==============================] - 0s 26ms/step - loss: 3.3095 - accuracy: 0.0000e+00
Epoch 9/50
2/2 [==============================] - 0s 16ms/step - loss: 3.2992 - accuracy: 0.0722
Epoch 10/50
2/2 [==============================] - 0s 26ms/step - loss: 3.2882 - accuracy: 0.0556
Epoch 11/50
2/2 [